In [1]:
import os
from pathlib import Path

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    competition = 'titanic'
    path = Path(f'/root/{competition}')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(competition, path=path)
        zipfile.ZipFile(f'{path}/titanic.zip').extractall(path)

In [2]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [3]:
!ls {path}

gender_submission.csv  test.csv  titanic.zip  train.csv


In [53]:
train_df = pd.read_csv(path/'train.csv')

In [47]:
test_df = pd.read_csv(path/'test.csv')

In [54]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='object')

In [50]:
test_df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='object')

In [51]:
test_df['Survived'] = np.NaN

In [55]:
train_df['train'] = 1
test_df['train'] = 0

In [56]:
df = pd.concat([train_df, test_df])

In [76]:
df['Famsize'] = df['SibSp'] + df['Parch'] + 1

In [57]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1


In [74]:
df.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
train             0
Sirname           0
Title             0
dtype: int64

In [58]:
df[df['Sex'] == 'male']['Survived'].mean()

0.18890814558058924

In [59]:
df[df['Sex'] == 'female']['Survived'].mean()

0.7420382165605095

In [60]:
df['Sirname'] = df['Name'].apply(lambda x: x.split(',')[0])

In [61]:
df.groupby('Sirname')['Survived'].mean().value_counts()

0.000000    387
1.000000    224
0.500000     31
0.666667     11
0.333333      7
0.750000      4
0.250000      2
0.222222      1
Name: Survived, dtype: int64

In [62]:
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [63]:
df['Title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: Title, dtype: int64

In [64]:
df[df['Title'] == 'Master'].head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train,Sirname,Title
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,NaN,S,1,Palsson,Master
16,17,0.0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.125,NaN,Q,1,Rice,Master


In [79]:
df[df['Sirname'] == 'Palsson']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train,Sirname,Title,Famsize
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,NaN,S,1,Palsson,Master,5
24,25,0.0,3,"Palsson, Miss. Torborg Danira",female,8.0,3,1,349909,21.075,NaN,S,1,Palsson,Miss,5
374,375,0.0,3,"Palsson, Miss. Stina Viola",female,3.0,3,1,349909,21.075,NaN,S,1,Palsson,Miss,5
567,568,0.0,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.075,NaN,S,1,Palsson,Mrs,5
389,1281,NaN,3,"Palsson, Master. Paul Folke",male,6.0,3,1,349909,21.075,NaN,S,0,Palsson,Master,5


In [77]:
df[df['Sirname'] == 'Rice']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train,Sirname,Title,Famsize
16,17,0.0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.125,NaN,Q,1,Rice,Master,6
171,172,0.0,3,"Rice, Master. Arthur",male,4.0,4,1,382652,29.125,NaN,Q,1,Rice,Master,6
278,279,0.0,3,"Rice, Master. Eric",male,7.0,4,1,382652,29.125,NaN,Q,1,Rice,Master,6
787,788,0.0,3,"Rice, Master. George Hugh",male,8.0,4,1,382652,29.125,NaN,Q,1,Rice,Master,6
885,886,0.0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.125,NaN,Q,1,Rice,Mrs,6
55,947,NaN,3,"Rice, Master. Albert",male,10.0,4,1,382652,29.125,NaN,Q,0,Rice,Master,6


In [71]:
male = (df['Sex'] == 'male')
survived = (df['Survived'] == 1)
df[(male) & (survived)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train,Sirname,Title
17,18,1.0,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,1,Williams,Mr
21,22,1.0,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0000,D56,S,1,Beesley,Mr
23,24,1.0,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S,1,Sloper,Mr
36,37,1.0,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C,1,Mamee,Mr
55,56,1.0,1,"Woolner, Mr. Hugh",male,NaN,0,0,19947,35.5000,C52,S,1,Woolner,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,839,1.0,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,NaN,S,1,Chip,Mr
839,840,1.0,1,"Marechal, Mr. Pierre",male,NaN,0,0,11774,29.7000,C47,C,1,Marechal,Mr
857,858,1.0,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S,1,Daly,Mr
869,870,1.0,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S,1,Johnson,Master


In [78]:
df[df['Sirname'] == 'Johnson']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train,Sirname,Title,Famsize
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,1,Johnson,Mrs,3
172,173,1.0,3,"Johnson, Miss. Eleanor Ileen",female,1.0,1,1,347742,11.1333,NaN,S,1,Johnson,Miss,3
302,303,0.0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0000,NaN,S,1,Johnson,Mr,1
597,598,0.0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0000,NaN,S,1,Johnson,Mr,1
719,720,0.0,3,"Johnson, Mr. Malkolm Joackim",male,33.0,0,0,347062,7.7750,NaN,S,1,Johnson,Mr,1
869,870,1.0,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S,1,Johnson,Master,3


In [80]:
df[df['Sirname'] == 'West']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train,Sirname,Title,Famsize
58,59,1.0,2,"West, Miss. Constance Mirium",female,5.00,1,2,C.A. 34651,27.75,NaN,S,1,West,Miss,4
450,451,0.0,2,"West, Mr. Edwy Arthur",male,36.00,1,2,C.A. 34651,27.75,NaN,S,1,West,Mr,4
472,473,1.0,2,"West, Mrs. Edwy Arthur (Ada Mary Worth)",female,33.00,1,2,C.A. 34651,27.75,NaN,S,1,West,Mrs,4
250,1142,NaN,2,"West, Miss. Barbara J",female,0.92,1,2,C.A. 34651,27.75,NaN,S,0,West,Miss,4


In [89]:
df['FamID'] = df['Sirname'] + df['Famsize'].astype(str)

In [113]:
def get_pred(row):
    is_male = row['Sex'] == 'male'
    is_boy = row['Title'] == 'Master'
    FamID = row['FamID']
    
    d_t = df[df['train'] == 1]
    if is_male:
        if is_boy:
            d_f = d_t[d_t['FamID'] == FamID]
            is_male = d_f['Sex'] == 'male'
            is_boy = d_f['Title'] == 'Master'
            if d_f[is_boy | ~is_male]['Survived'].mean() == 1:
                return 1  
            else:
                return 0
        else:
            return 0
    else:
        d_f = d_t[d_t['FamID'] == FamID]
        is_male = d_f['Sex'] == 'male'
        is_boy = d_f['Title'] == 'Master'
        if d_f[is_boy | ~is_male]['Survived'].mean() == 0:
            return 0
        else:
            return 1
        
    return None


is_test = df['train'] == 0
test = df[is_test].copy()
test['Survived'] = test.apply(lambda x: get_pred(x), axis=1)

In [115]:
test[['PassengerId', 'Survived']].to_csv('sub_hack.csv', index=False)